In [13]:
# For following a second Drone
#Must have second drone in sights of Mambo Drone
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import random
import tensorflow as tf
from sklearn.utils import shuffle



In [2]:
import pickle
import zipfile 

In [3]:
training_file = "./traffic-signs-data/train.p"
validation_file= "./traffic-signs-data/valid.p"
testing_file = "./traffic-signs-data/valid.p"

with open(training_file, mode = 'rb') as f:
    train = pickle.load(f)

with open(testing_file, mode = 'rb') as f:
    test = pickle.load(f)
    
with open(validation_file, mode = 'rb') as f:
    valid = pickle.load(f)
    

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
X_valid, y_test = valid['features'], valid['features']


In [4]:
import numpy as np

n_train = len(X_train)
n_test = len(X_test)
n_valid = len(X_valid)

#Image shape and dimensions
image_shape = X_train[0].shape
#Number of classes and labels

n_classes = len(np.unique(y_train))

print("Number of training examples = ", n_train)
print("Number of testing examples = ", n_test)
print("Image shape = ", image_shape)
print("Number of classes = ", n_classes)

Number of training examples =  34799
Number of testing examples =  4410
Image shape =  (32, 32, 3)
Number of classes =  43


In [6]:
#Preprocess the Dataset 

#Grayscale 

X_train_gray = np.sum(X_train / 3, axis = 3, keepdims=True)
X_valid_gray = np.sum(X_valid / 3, axis = 3, keepdims=True)
X_test_gray = np.sum(X_test / 3, axis = 3, keepdims= True)
print("X_train_gray", X_train_gray.shape)

X_train_gray (34799, 32, 32, 1)


In [10]:
# Normalize images 

X_train_normalize = ((X_train_gray - 128) / 128)
X_test_normalize = ((X_test_gray - 128) / 128)

print(np.mean(X_train_normalize))

X_train = X_train_normalize
X_test = X_test_normalize

-0.354081335648


In [11]:
#Image Augmentation Functions 
import skimage

In [14]:
def sharpen_image(img):
    gb = cv2.GuassianBlur(img, (5,5), 20.0)
    return cv2.addWeighted(img, 2, gb, -1, 0)

In [15]:
def translate_random(img):
    np.random.seed(43)
    rand_x = np.random.randit(-5, 5)
    rand_y = np.random.randit(-5, 5)
    translation_matrix = np.float32([ [1,0,rand_x], [0,1,rand_y]])
    return cv2.warpAffine(img, translation_matrix, (32, 32))

In [16]:
def add_noise(img):
    noisy_image = skimage.util.random_noise(img, mode='gaussian', seed=None, clip=True) 
    return np.asarray(noisy_image, dtype="uint8")

In [17]:
def augment_all(data):
    for x in data:
        rand_number = random.randit(0, 6)
        if rand_number == 0 or rand_number == 1:
            x = add_noise(x)
        if rand_number == 2:
            x = translate_random(x)
        if rand_number == 3:
            x = sharpen_image(x)
    return data

In [18]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers import Dropout
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.
